# 柔顺度计算

In [1]:
from compliance_minimizer_copy import ComplianceMinimizer
import jax.numpy as jnp 

Opt = ComplianceMinimizer()

rho = jnp.ones((Opt.mesh['nelx'] * Opt.mesh['nely']))
compliance = Opt.computeCompliance(rho)
print("Compliance with default rho:", compliance)

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[]..
The error occurred while tracing the function projectionFilter at /home/heliang/FEALPy_Development/fealpy/app/StruTopoOpt/compliance_minimizer_copy.py:6 for jit. This concrete value was not available in Python because it depends on the value of the argument projection['isOn'].
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

JAX 遇到 TracerBoolConversionError 的原因是因为 JAX 无法在 JIT 编译的函数内部对布尔值进行直接判断。这个错误源自于 projection['isOn'] 的布尔判断。

## 投影滤波器

In [ ]:
def test_projectionFilter(rho, beta, c0, isOn):
    if isOn:
        v1 = jnp.tanh(c0 * beta)
        nm = v1 + jnp.tanh(beta * (rho - c0))
        dnm = v1 + jnp.tanh(beta * (1. - c0))
        return nm / dnm
    else:
        return rho
rho_values = jnp.array([0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0])
# 投影参数
beta = 4
c0 = 0.5
isOn = True
# 打印原始 rho 值和经过投影滤波器后的 rho 值
print("Original rho values: ", rho_values)
projected_rho_values = test_projectionFilter(rho_values, beta, c0, isOn)
print("Projected rho values: ", projected_rho_values.round(3))

Original rho values:  [0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.   ]
Projected rho values:  [0.         0.031      0.105      0.26000002 0.5        0.74
 0.89500004 0.96900004 1.        ]


## 测试单元刚度矩阵

In [ ]:
KE = Opt.K0
print("KE:\n", KE)

KE:
 [[ 0.4945055   0.17857143 -0.3021978  -0.01373626 -0.24725275 -0.17857143
   0.05494506  0.01373626]
 [ 0.17857143  0.4945055   0.01373626  0.05494506 -0.17857143 -0.24725275
  -0.01373626 -0.3021978 ]
 [-0.3021978   0.01373626  0.4945055  -0.17857143  0.05494506 -0.01373626
  -0.24725275  0.17857143]
 [-0.01373626  0.05494506 -0.17857143  0.4945055   0.01373626 -0.3021978
   0.17857143 -0.24725275]
 [-0.24725275 -0.17857143  0.05494506  0.01373626  0.4945055   0.17857143
  -0.3021978  -0.01373626]
 [-0.17857143 -0.24725275 -0.01373626 -0.3021978   0.17857143  0.4945055
   0.01373626  0.05494506]
 [ 0.05494506 -0.01373626 -0.24725275  0.17857143 -0.3021978   0.01373626
   0.4945055  -0.17857143]
 [ 0.01373626 -0.3021978   0.17857143 -0.24725275 -0.01373626  0.05494506
  -0.17857143  0.4945055 ]]


# 测试体积约束
体积约束
$$g \equiv \frac{\sum_e\rho_ev_e}{V^*} - 1 \leq 0$$
其中 $\rho_e$ 和 $v_e$ 分别是第 $e$ 个单元的密度和体积。

`computeGlobalVolumeConstraint` 中的实现假设了每个单元体积相同且归一化，从而使用单元密度的平均值 $\mathrm{mean}(\rho)$ 来近似 $\sum_e\rho_ev_e$

对于输入 $\rho=[0.1,0.3,0.5,0.7,0.9]$，单元体积 $v_e=0.2$，因此 $g(\rho)=\frac{0.5}{0.5}-1=0$，$\frac{\partial{g(\rho)}}{\partial\rho}=\frac{v_e}{V^*}=\frac{0.2}{0.5}=0.4$

In [ ]:
import jax.numpy as jnp
from jax import value_and_grad

# 定义计算体积约束的函数
def computeGlobalVolumeConstraint(rho, vf):
    g = jnp.mean(rho) / vf - 1.
    return g

# 测试函数
def test_computeConstraints(rho, vf):
    # 计算体积约束的值及其灵敏度
    constraint_value, constraint_grad = value_and_grad(computeGlobalVolumeConstraint)(rho, vf)
    return constraint_value, constraint_grad

# 数值梯度计算函数
def numerical_grad(f, x, vf, eps=1e-5):
    grad = jnp.zeros_like(x)
    for i in range(len(x)):
        x_pos = x.at[i].add(eps)
        x_neg = x.at[i].add(-eps)
        grad = grad.at[i].set((f(x_pos, vf) - f(x_neg, vf)) / (2 * eps))
    return grad

# 设置测试的 rho 值和体积分数 vf
rho_values = jnp.array([0.1, 0.3, 0.5, 0.7, 0.9])
vf = 0.5

# 打印原始 rho 值
print("Original rho values: ", rho_values)

# 调用测试函数并打印结果
constraint_value, constraint_grad = test_computeConstraints(rho_values, vf)
print("test:", jnp.mean(rho_values))
print("Constraint value: ", constraint_value)
print("Constraint gradient: ", constraint_grad)

# 计算数值梯度
numerical_gradient = numerical_grad(computeGlobalVolumeConstraint, rho_values, vf)
print("Numerical gradient: ", numerical_gradient)

# 比较数值梯度和自动微分计算的梯度
error = jnp.linalg.norm(constraint_grad - numerical_gradient)
print("Gradient error: ", error)


Original rho values:  [0.1 0.3 0.5 0.7 0.9]
test: 0.5
Constraint value:  0.0
Constraint gradient:  [0.4 0.4 0.4 0.4 0.4]
Numerical gradient:  [0.40233135 0.40233135 0.40233135 0.40233135 0.40233135]
Gradient error:  0.005213049
